In [ ]:
!pip install opencv-python

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import skimage as sk
import skimage.io as skio
from scipy.signal import convolve2d
from skimage import io, color
import skimage.transform as sktr
import cv2

plt.rcParams['figure.figsize'] = [8, 8]

In [ ]:
def saveImage(img, name):
    skio.imsave("output/" + name + ".jpg", img)

## Part 1: Fun with Filters

### Part 1.1: Finite Difference Operator

In [ ]:
name = "data/cameraman.png"
cameraman_img = skio.imread(name, as_gray=True)
cameraman_img = cv2.normalize(cameraman_img.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
skio.imshow(cameraman_img)
skio.show()

In [ ]:
# Difference Operators Convolution
D_x = np.tile(np.array([-1, 0, 1])[None,:], [3, 1]) / 2
D_y = np.tile(np.array([-1, 0, 1])[:, None], [1, 3]) / 2

partial_x = convolve2d(cameraman_img, D_x, mode='same')
partial_y = convolve2d(cameraman_img, D_y, mode='same')

# Computing Gradiant Magnitude
grad_mag = np.sqrt(partial_x**2 + partial_y**2)

# Binarizing Gradient Magnitude
threshold = 0.1 * np.max(grad_mag)
grad_mag_bin = np.where(grad_mag > threshold, 1, 0)

plt.figure(figsize=(10, 30))
plt.subplot(4, 1, 1)
plt.imshow(partial_x, cmap='gray')
plt.title('Partial Derivative in X')
plt.colorbar()
saveImage(partial_x, "cameraman_1_partialX")

plt.subplot(4, 1, 2)
plt.imshow(partial_y, cmap='gray')
plt.title('Partial Derivative in Y')
plt.colorbar()
saveImage(partial_y, "cameraman_1_partialY")

plt.subplot(4, 1, 3)
plt.imshow(grad_mag, cmap='gray')
plt.title('Gradient Magnitude')
plt.colorbar()
saveImage(grad_mag, "cameraman_1_gradMag")

plt.subplot(4, 1, 4)
plt.imshow(grad_mag_bin, cmap='gray')
plt.title('Gradient Magnitude Binarized')
plt.colorbar()
saveImage(grad_mag_bin, "cameraman_1_gradMagBin")


### Part 1.2: Derivative of Gaussian Filter

In [ ]:
# Generating Gaussian Filter
k_size = 9
sigma = 0
gauss = cv2.getGaussianKernel(k_size, sigma)
G = gauss * gauss.T

# Convolving Image with Gaussian
blurred_img = convolve2d(cameraman_img, G, mode='same')
saveImage(blurred_img, "cameraman_2_blurred")

# Repeating 1.1 Process

# Difference Operators Convolution
D_x = np.tile(np.array([-1, 0, 1])[None,:], [3, 1]) / 2
D_y = np.tile(np.array([-1, 0, 1])[:, None], [1, 3]) / 2

partial_x = convolve2d(blurred_img, D_x, mode='same')
partial_y = convolve2d(blurred_img, D_y, mode='same')

# Computing Gradiant Magnitude
grad_mag = np.sqrt(partial_x**2 + partial_y**2)

# Binarizing Gradient Magnitude
threshold = 0.12
grad_mag_bin = np.where(grad_mag > threshold, 1, 0)

plt.figure(figsize=(10, 30))
plt.subplot(4, 1, 1)
plt.imshow(partial_x, cmap='gray')
plt.title('Partial Derivative in X')
plt.colorbar() 
saveImage(partial_x, "cameraman_2_partialX")

plt.subplot(4, 1, 2)
plt.imshow(partial_y, cmap='gray')
plt.title('Partial Derivative in Y')
plt.colorbar()
saveImage(partial_y, "cameraman_2_partialY")

plt.subplot(4, 1, 3)
plt.imshow(grad_mag, cmap='gray')
plt.title('Gradient Magnitude')
plt.colorbar()
saveImage(grad_mag, "cameraman_2_gradMag")

plt.subplot(4, 1, 4)
plt.imshow(grad_mag_bin, cmap='gray')
plt.title('Gradient Magnitude Binarized')
plt.colorbar()
saveImage(grad_mag_bin, "cameraman_2_gradMagBin")


In [ ]:
# Convolving Gaussian with D_x and D_y
DoG_x = convolve2d(G, D_x, mode='same')
DoG_y = convolve2d(G, D_y, mode='same')

saveImage(DoG_x, "cameraman_3_DOGX")
saveImage(DoG_y, "cameraman_3_DOGY")

partial_DoG_x = convolve2d(cameraman_img, DoG_x, mode='same')
partial_DoG_y = convolve2d(cameraman_img, DoG_y, mode='same')

# Computing Gradiant Magnitude
grad_mag = np.sqrt(partial_DoG_x**2 + partial_DoG_y**2)

# Binarizing Gradient Magnitude
threshold = 0.12
grad_mag_bin = np.where(grad_mag > threshold, 1, 0)

plt.figure(figsize=(10, 30))
plt.subplot(4, 1, 1)
plt.imshow(partial_DoG_x, cmap='gray')
plt.title('Partial Derivative in X DoG Filter')
plt.colorbar()
saveImage(partial_DoG_x, "cameraman_3_partialDOGX")

plt.subplot(4, 1, 2)
plt.imshow(partial_DoG_y, cmap='gray')
plt.title('Partial Derivative in Y DoG Filter')
plt.colorbar()
saveImage(partial_DoG_y, "cameraman_3_partialDOGY")

plt.subplot(4, 1, 3)
plt.imshow(grad_mag, cmap='gray')
plt.title('Gradient Magnitude')
plt.colorbar()
saveImage(grad_mag, "cameraman_3_gradMag")

plt.subplot(4, 1, 4)
plt.imshow(grad_mag_bin, cmap='gray')
plt.title('Gradient Magnitude Binarized')
plt.colorbar()
saveImage(grad_mag_bin, "cameraman_3_gradMagBin")

## Part 2: Fun with Frequencies

### Part 2.1: Image "Sharpening"

In [ ]:
def unsharp_mask_filter(image, name, k_size, sigma, alpha):
    image = cv2.normalize(image.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    
    plt.imshow(image)
    plt.show()
    
    # creating Guassian filter
    gauss = cv2.getGaussianKernel(k_size, sigma)
    G = np.outer(gauss, gauss.T)
    
    # blurring image
    r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    blur_r = convolve2d(r, G, mode='same')
    blur_g = convolve2d(g, G, mode='same')
    blur_b = convolve2d(b, G, mode='same')
    blur = np.stack([blur_r, blur_g, blur_b], axis=-1)
    
    plt.imshow(blur)
    plt.show()
    saveImage(blur, name + "_blur")
    
    gray_image = color.rgb2gray(image)
    gray_blur = color.rgb2gray(blur)
    high_freq = gray_image - gray_blur
    
    plt.imshow(high_freq, cmap="gray")
    plt.show()
    saveImage(high_freq, name + "_high_freq")
    
    sharp_r = np.clip(image[:, :, 0] + (alpha * high_freq), 0, 1)
    sharp_g = np.clip(image[:, :, 1] + (alpha * high_freq), 0, 1)
    sharp_b = np.clip(image[:, :, 2] + (alpha * high_freq), 0, 1)
    sharp = np.stack([sharp_r, sharp_g, sharp_b], axis=-1)
    
    plt.imshow(sharp)
    plt.show()
    saveImage(sharp, name + "_sharp")
    
    return sharp

name = "data/taj.jpg"
taj_img = skio.imread(name)
sharp_taj = unsharp_mask_filter(taj_img, "taj", 9, 2, 2)

In [ ]:
sharp_taj2 = unsharp_mask_filter(sharp_taj, "sharp_taj", 9, 2, 2)

In [ ]:
circuit_img = skio.imread("data/circuit.jpg")
sharp_circuit = unsharp_mask_filter(circuit_img, "circuit", 9, 2, 2)

In [ ]:
gateway_img = skio.imread("data/gateway.jpg")
sharp_gateway = unsharp_mask_filter(gateway_img, "gateway", 9, 2, 2)

In [ ]:
#vt_img = skio.imread("data/victoria_station.jpg")
#sharp_vt = unsharp_mask_filter(vt_img, 55, 0, 2)

In [ ]:
temple_img = skio.imread("data/temple.jpg")
sharp_temple = unsharp_mask_filter(temple_img, "temple", 9, 2, 2)

### Part 2.2: Hybrid Images

In [ ]:
def get_points(im1, im2):
    print('Please select 2 points in each image for alignment.')
    fig, ax = plt.subplots()
    plt.imshow(im1)
    p1, p2 = plt.ginput(2)
    plt.close()
    plt.imshow(im2)
    p3, p4 = plt.ginput(2)
    plt.pause(1)
    plt.close(fig)
    return (p1, p2, p3, p4)

def recenter(im, r, c):
    R, C, _ = im.shape
    rpad = (int) (np.abs(2*r+1 - R))
    cpad = (int) (np.abs(2*c+1 - C))
    return np.pad(
        im, [(0 if r > (R-1)/2 else rpad, 0 if r < (R-1)/2 else rpad),
             (0 if c > (C-1)/2 else cpad, 0 if c < (C-1)/2 else cpad),
             (0, 0)], 'constant')

def find_centers(p1, p2):
    cx = np.round(np.mean([p1[0], p2[0]]))
    cy = np.round(np.mean([p1[1], p2[1]]))
    return cx, cy

def align_image_centers(im1, im2, pts):
    p1, p2, p3, p4 = pts
    h1, w1, b1 = im1.shape
    h2, w2, b2 = im2.shape
    
    cx1, cy1 = find_centers(p1, p2)
    cx2, cy2 = find_centers(p3, p4)

    im1 = recenter(im1, cy1, cx1)
    im2 = recenter(im2, cy2, cx2)
    return im1, im2

def rescale_images(im1, im2, pts):
    p1, p2, p3, p4 = pts
    len1 = np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2)
    len2 = np.sqrt((p4[1] - p3[1])**2 + (p4[0] - p3[0])**2)
    dscale = len2/len1
    if dscale < 1:
        im1 = sktr.rescale(im1, (dscale, dscale, 1))
    else:
        im2 = sktr.rescale(im2, (1./dscale, 1./dscale, 1))
    return im1, im2

def rotate_im1(im1, im2, pts):
    p1, p2, p3, p4 = pts
    theta1 = math.atan2(-(p2[1] - p1[1]), (p2[0] - p1[0]))
    theta2 = math.atan2(-(p4[1] - p3[1]), (p4[0] - p3[0]))
    dtheta = theta2 - theta1
    im1 = sktr.rotate(im1, dtheta*180/np.pi)
    return im1, dtheta

def match_img_size(im1, im2):
    # Make images the same size
    h1, w1, c1 = im1.shape
    h2, w2, c2 = im2.shape
    if h1 < h2:
        im2 = im2[int(np.floor((h2-h1)/2.)) : -int(np.ceil((h2-h1)/2.)), :, :]
    elif h1 > h2:
        im1 = im1[int(np.floor((h1-h2)/2.)) : -int(np.ceil((h1-h2)/2.)), :, :]
    if w1 < w2:
        im2 = im2[:, int(np.floor((w2-w1)/2.)) : -int(np.ceil((w2-w1)/2.)), :]
    elif w1 > w2:
        im1 = im1[:, int(np.floor((w1-w2)/2.)) : -int(np.ceil((w1-w2)/2.)), :]
    assert im1.shape[0] == im2.shape[0] and im1.shape[1] == im2.shape[1]
    return im1, im2

def align_images(im1, im2):
    %matplotlib tk
    pts = get_points(im1, im2)
    %matplotlib inline
    im1, im2 = align_image_centers(im1, im2, pts)
    im1, im2 = rescale_images(im1, im2, pts)
    im1, angle = rotate_im1(im1, im2, pts)
    im1, im2 = match_img_size(im1, im2)
    return im1, im2

def hybrid_image(im1, im2, sigma1, sigma2):
    im1_gray, im2_gray = color.rgb2gray(im1), color.rgb2gray(im2)
    k_size1 = int(6 * sigma1 + 1)
    k_size1 += 1 - k_size1 % 2  
    k_size2 = int(6 * sigma2 + 1)
    k_size2 += 1 - k_size2 % 2  
    
    gauss1 = cv2.getGaussianKernel(k_size1, sigma1)
    G1 = np.outer(gauss1, gauss1.T)
    im1_low = convolve2d(im1_gray, G1, mode='same')
    high = im1_gray - im1_low
    
    gauss2 = cv2.getGaussianKernel(k_size2, sigma2)
    G2 = np.outer(gauss2, gauss2.T)
    low = convolve2d(im2_gray, G2, mode='same')
    
    return (low + high), low, high

In [ ]:
def hybrid_images_main(im1, im2, sigma1, sigma2):
    # Next align images (this code is provided, but may be improved)
    im1_aligned, im2_aligned = align_images(im1, im2)

    ## You will provide the code below. Sigma1 and sigma2 are arbitrary 
    ## cutoff values for the high and low frequencies

    hybrid, low, high = hybrid_image(im1_aligned, im2_aligned, sigma1, sigma2)

    plt.imshow(hybrid, cmap="gray")
    plt.show
    return hybrid, low, high


In [ ]:
im1 = plt.imread('hybrid_python/nutmeg.jpg')/255
im2 = plt.imread('hybrid_python/DerekPicture.jpg')/255.
hybrid, low, high = hybrid_images_main(im1, im2, 20, 10)
saveImage(hybrid, "nutmegXderek")
saveImage(low, "derekLow")
saveImage(high, "nutmegHi")

In [ ]:
im2 = plt.imread('hybrid_python/mop_dog.jpg')/255
im1 = plt.imread('hybrid_python/kanye.jpg')/255
hybrid, low, high = hybrid_images_main(im1, im2, 10, 5)
saveImage(hybrid, "dogXkanye")
saveImage(low, "dogLow")
saveImage(high, "kanyeHi")

In [ ]:
im2 = plt.imread('hybrid_python/tiger.jpg')/255
im1 = plt.imread('hybrid_python/wolf.jpg')/255.
hybrid, low, high = hybrid_images_main(im1, im2, 3, 3)
saveImage(hybrid, "wolfXtiger")
saveImage(low, "tigerLow")
saveImage(high, "wolfHi")

In [ ]:
# FFT
## FFT images
ffim1g = np.log(np.abs(np.fft.fftshift(np.fft.fft2(color.rgb2gray(im1)))))
ffim2g = np.log(np.abs(np.fft.fftshift(np.fft.fft2(color.rgb2gray(im2)))))
fflow = np.log(np.abs(np.fft.fftshift(np.fft.fft2(low))))
ffhigh = np.log(np.abs(np.fft.fftshift(np.fft.fft2(high))))
ffhybrid = np.log(np.abs(np.fft.fftshift(np.fft.fft2(hybrid))))
plt.figure()
plt.imshow(ffim1g)
saveImage(ffim1g, "wolf_ff")

plt.figure()
plt.imshow(ffim2g)
saveImage(ffim2g, "tiger_ff")

plt.figure()
plt.imshow(fflow)
saveImage(fflow, "tiger_low_ff")

plt.figure()
plt.imshow(ffhigh)
saveImage(ffhigh, "wolf_high_ff")

plt.figure()
plt.imshow(ffhybrid)
plt.show()
saveImage(ffhybrid, "tigerXwolf_ff")

In [ ]:
#TODO: Find Better Images
im1 = plt.imread('hybrid_python/running.jpg')/255
im2 = plt.imread('hybrid_python/walking.jpg')/255.
hybrid, low, high = hybrid_images_main(im1, im2, 10, 1)
saveImage(hybrid, "runningXwalking")
saveImage(low, "walkingLow")
saveImage(high, "runningHi")

### Part 2.3: Gaussian and Laplacian Stacks

In [ ]:
def gaussianStack(im, k_size, sigma, N):
    im = color.rgb2gray(im)
    im = cv2.normalize(im.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    gauss = cv2.getGaussianKernel(k_size, sigma)
    G = np.outer(gauss, gauss.T)
    # x, y, _ = im.shape
    # stack = np.zeros((x, y, N))
    stack = [im]
    for i in range(1, N):
        curr_im = stack[-1]
        image = convolve2d(curr_im, G, mode='same')
        image = cv2.normalize(image.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
        image = color.rgb2gray(image)
        stack.append(image)
        
    return stack

def laplacianStack(gstack):
    lstack = []
    for i in range(0, len(gstack) - 1):
        image = gstack[i] - gstack[i + 1]
        lstack.append(image)
        if i == (len(gstack) - 2):
            lstack.append(gstack[i + 1])
    
    return lstack

def showStack(stack, name, save=True):
    for idx, image in enumerate(stack):
        plt.imshow(image, cmap="gray")
        plt.show()
        if save:
            saveImage(image, name+str(idx))

In [ ]:
im = plt.imread('data/oraple.jpg')/255
gstack = gaussianStack(im, 15, 5, 10)
showStack(gstack, "oraple_gs")

In [ ]:
lstack = laplacianStack(gstack)
showStack(lstack, "oraple_ls")

### Part 2.4: Multiresolution Blending

In [ ]:
def create_mask(h, w, split, im, blend_type="vertical"):
    mask = np.zeros((h, w))
    if blend_type == "vertical":
        mask[:, :split] = 1
    elif blend_type == "horizontal":
        mask[:split, :] = 1
    # elif blend_type == "irregular":
      #   threshold = 0.7
      #   mask = np.where(im > threshold, 0, 1).astype(np.uint8)
    
    return mask

def blend(im1, im2, blend_type, N, irreg_mask, name):
    if not im1.shape == im2.shape:
        im1, im2 = match_img_size(im1, im2)
        
    # Creating Laplacian stacks
    im1, im2 = color.rgb2gray(im1), color.rgb2gray(im2)
    im1gStack = gaussianStack(im1, 10, 5, N)
    im1lStack = laplacianStack(im1gStack)
    im2gStack = gaussianStack(im2, 10, 5, N)
    im2lStack = laplacianStack(im2gStack)
    
    # Creating mask
    h, w = im1.shape
    if blend_type == "vertical":
        mask = create_mask(h, w, w // 2, None, blend_type)
    elif blend_type == "horizontal":
        mask = create_mask(h, w, h // 2, None, blend_type)
    else:
        threshold = 0.5
        mask = np.where(color.rgb2gray(irreg_mask) > threshold, 1, 0).astype(np.uint8)
        mask = mask[:h, :w]
    
    saveImage(mask, "mask_" + blend_type)
    
    # Extra Gaussian Blurring for 1st layer
    gauss = cv2.getGaussianKernel(30, 10)
    G = np.outer(gauss, gauss.T)
    mask = convolve2d(mask, G, mode='same')
    mask = cv2.normalize(mask.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    
    gmask = gaussianStack(mask, 30, 10, N)
    
    showStack(gmask, name, False)
    
    # Blending
    blend = np.zeros((h, w))
    for i in range(0, N):
        left = gmask[i] * im1lStack[i]
        saveImage(left, name + "_left_" + str(i))
        showImage(left)
        right = (1 - gmask[i]) * im2lStack[i]
        saveImage(right, name + "_right_" + str(i))
        showImage(right)
        blend += left + right
        saveImage(blend, name + "_blend_" + str(i))
        
    showImage(blend)
    saveImage(blend, name + "_blend_final")
    return blend
    
def showImage(im):
    plt.imshow(im, cmap="gray")
    plt.show()

In [ ]:
im1 = plt.imread("spline/apple.jpeg")/255
im2 = plt.imread("spline/orange.jpeg")/255

blended = blend(im1, im2, "vertical", 10, None, "oraple")

In [ ]:
im1 = plt.imread("spline/fire.jpg")/255
im2 = plt.imread("spline/ocean.jpg")/255

blended = blend(im1, im2, "horizontal", 10, None, "ficean")

In [ ]:
im1 = plt.imread("spline/trump.jpg")/255
im2 = plt.imread("spline/space.jpg")/255
mask = plt.imread("spline/mask.jpg")/255

blended = blend(im1, im2, "irregular", 10, mask, "spaceTrump")